# Module 1 - Day 4: Polyelectronic atoms. Electronic correlation and self-consistent field methods

## Learning objectives:

* Explain the "many-body" problem in terms of how the Schrodinger equation for polyelectronic atoms has a term that does not allow us to separate the problem into independent electrons and therefore obtaining an exact solution.
* The Hartree-Fock method: Explain variational and mean-field approaches to the solution and the need for a basis set. Explain the Self Consistent Field method.
* Understand and predict the possible spin multiplicity in an atom
* Using pyscf package to calculate ground states of atoms and other excited spin states.
* Using pyscf package to calculate ionization energies
* Learning to read and write files with Python


# The Hamiltonian for polyelectronic atoms

$$
\hat{H} = \text{Kinetic Energy} + \text{Potential Energy}
$$
For atoms in atomic units:

$$
\hat{H} = -\frac{1}{2}\nabla_{nucleus} -\frac{1}{2}\sum_{i=elec} \nabla_i + \sum_{i=elec} \frac{q_{i}q_{nucl}}{r_{i}} + \sum_{i,j=elec} \frac{q_{i}q_{j}}{r_{ij}}
$$

Translated into English we have:

$$
\hat{H} = \text{Kinetic of nucleus} + \sum{\text{Kinetic of e-}} + \sum{\text{attraction of elec-nucl}} + \sum{\text{Repulsion elec-elect}}
$$

Notice that if it weren't for the last term, the electron-electron repulsion, our Hamiltonian could be written just like a sum of hydrogen-like hamiltonians (independent electrons $\hat{h}_i$)

$$
\hat{H} = \sum_{i=elec} \hat{h}_i + \sum_{i,j=elec} \frac{q_{i}q_{j}}{r} 
$$



**The good news**: if we could ignore the repulsion e-e ($\sum_{i,j=elec} \frac{q_{i}q_{j}}{r_{ij}}$) we could use the hydrogen atom solutions (1s, 2s, 2p...).
Algebra of operators tell us that sum of operators give product of eigenfunctions and sum of energies. In other words, **IF** we ignore the electron repulsions:

* $\hat{H} = \sum_{i=elec} \hat{h}_i$
* The wavefunction is what we know as electronic configurations: $\Psi=\phi_{1s}\phi_{2s}\phi_{2p}...$
* The energy is the sum of each electron on separate orbitals: $E=E(1s) + E(2s) + E(2p)$

**The bad news**: While we may still use the electronic configuration (1s2s2p...) to rationalize a lot of atom behavior in chemistry, the energy contribution of the electronic repulsion is too large to obtain any quantitative energetic study.

This problem is yet another example of what is known as [the many-body problem](https://en.wikipedia.org/wiki/Many-body_problem) where an interacting system cannot be simplified as a sum of its parts.

 Another complication is that electrons because they are fermions (fractional spin) cannot be expressed just as a linear combination of basis, they need to be described with a determinant ([Slater determinant](https://en.wikipedia.org/wiki/Slater_determinant)). So, just keep in mind that in the above equation $\Psi$ cannot just be a product of functions, it has to be a *symmetrized product*. Nothing to worry about, but it will bring two types of electron repulsions, the coulomb repulsions that we already know but also the fermionic ones. Stay tuned.
 

# The Self-Consistent Field solution (SCF) or Hartree-Fock


A very common approach to many-body problems like polyelectronic atoms is the use the Variational method. In this case, this method tell us that any function that we try for our Hamiltonian has an expectation value of the energy that is an upper bound to the ground state energy of the system. We could just do some trial-and-error, put any function in the Hamiltonian and see which one gives us the lowest value. A more elegant way is to minimize the energy using the [Lagrange Multiplier Method](https://en.wikipedia.org/wiki/Lagrange_multiplier)

We still need to start with a function, a guess.

## The need for a base

As we said above, the hydrogen atomic orbitals (1s,2s,2p...) are not the solution, but they can be used as a beginning! We can use what is called in linear algebra a [base](https://en.wikipedia.org/wiki/Basis_(linear_algebra)). 

You know that a 3D vector can be expressed in terms of the base $\overrightarrow{i}, \overrightarrow{j}, \overrightarrow{k}$. So:

$$
(2,-3,3) = 2\cdot\overrightarrow{i} - 3\cdot\overrightarrow{j} + 3\cdot\overrightarrow{k}
$$

or more generally we can generate any vector $\overrightarrow{v}$ as a linear combination of a base $\{u_i\}$ and its coefficients $\{u_i\}$.

$$
\overrightarrow{v} = \sum_{i} c_i\cdot u_i
$$

We can express the wavefunction that we need to know as a combination of functions (bases) that we already know 

$$
\Psi = \sum_i c_i \cdot \phi_i
$$

The problem is now numerical a numerical one, this is . We need to find the best coefficients $\{c_i\}$ that give us the lowest possible energy.

We are skipping many steps but what is important is that 
1. The choice of a base is important
2. The process will be iterative

This whole process is called the [Hartree-Fock method](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)

1. Suggest a base (a collection of functions), a charge (number of electrons) and spin (number of unpaired electrons)
1. Give a guess to the coefficients $\{c_i\}$
2. Build the Fock matrix
3. Because our base is not orthogonal we have to carry what is called an overlap matrix which takes care of the orthonormalization of our base.
3. Diagonzalize the matrices.
4. Find out if the energy has lowered enough from the previous point. That is, if $\Delta E$ is lower than a threshold. If not, suggest a new change to $\{c_i\}$ and go back to point 2.

# Quantum chemistry packages: pyscf



The [PySCF](https://pyscf.org/) is a quantum chemistry package that will take care of all the calculations. We will have to input the following information:

* Geometry of the system in xyz coodinates. For one atom of course we can just place it in (0,0,0)
* Identity of the atoms that compose our system: H, He, Li...
* Charge of the system, if it's neutral the charge is zero.
* Spin: the number of unpaired electrons in the electronic configuration that you want to calculate. When there are unpaired electrons we will use the UHF method.

Before you run the scripts below, make sure you have installed the pyscf libraries

## Single point calculations. Reading the output

We are going to run a single-point calculation of the energy for the atom of lithium.
Notice that the basis set used here is 'sto-3g' which is the smallest, also known minimimal basis set where each orbital is represented by one function.

Pay attention to the geometry:
* charge is zero
* spin in this case refers to the number of unpaired electrons
* one single atom can be in the center of the xyz space: x=0, y=0, z=0.

The set_options are:
* basis is sto-3g
* reference uhf because of unpaired electrons (use rhf for all paired electrons)
* [scf_type](https://psicode.org/psi4manual/master/autodir_options_c/globals__scf_type.html) specifies one specific convergece algorithm.

In [ ]:
from pyscf import gto, scf

mol = gto.M(atom="Li 0 0 0", 
            basis="sto-3g", 
            charge=0, 
            spin=1,# spin=1 for one unpaired electron: 1s2 2s1
            verbose=5,
            output='m1d4_lithium.out')  #you can write a verbose output into a file



li = scf.UHF(mol) #Using UHF for unpaired electrons instead of RHF for paired electrons
li.kernel()
# Orbital energies, Mulliken population etc.
li.analyze()

# Access information about the calculation
print("SCF Converged:", li.converged)
print("Total Energy:", li.e_tot)

The output in the previous cell is so large that it masks the line where we print the energy.
We are now going to run the same calculation. However, we will write the results in a file that will appear in the same folder as where this notebook is.

Pay special attention to the code below. It opens the file and reads it line by line until it finds the line where the "Total Energy" is written.

In [ ]:
#myoutput will be an array containing the lines of the file
myoutput = open('m1d4_lithium.out','r').readlines()

for line in myoutput:
    #print(line)
    if 'Total Energy' in line:
        ene = line
print(ene)

Your turn: 
* How many iterations of the "Self Consistent Field" did it take to converge the energy?
* What is the energy of the 1s and 2s orbitals?

## Testing basis sets

The [variational method](https://en.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) tell us that the larger the basis set the better the result is. Since we know that the energy of hydrogen is -0.5 hartrees we can test this assumption by changing the basis set.

https://github.com/pyscf/pyscf/blob/master/examples/gto/04-input_basis.py 

In [ ]:

# ==> Compute H atom energy with Hartree-Fock using pyscf <==

# the H atom has a charge of 0, and one unpaired electron
# and we place it at the xyz origin (0,0,0)


basis = ["sto3g","3-21g","6-31g","ccpvdz"]
for base in basis:
    mol = gto.M(atom="H 0 0 0", 
        basis=base, 
        charge=0, 
        spin=1,# spin=1 for one unpaired electron: 1s1
        verbose=4,
        output='m1d4_H_'+base+'.out')  
    h = scf.UHF(mol).run() #Using UHF for unpaired electrons instead of RHF for paired electrons
    print(h.e_tot)



## Different levels of theory



In [ ]:
from pyscf import dft
def calc_H_level(level,basisset,myatom,mycharge,myspin):
    mol = gto.M(atom=myatom+" 0 0 0", 
        basis=basisset, 
        charge=mycharge, 
        spin=myspin,# spin=1 for one unpaired electron: 1s2 2s1
        verbose=4,
        output='m1d4_'+myatom+'_'+str(mycharge)+'_'+basisset+'_'+level+'.out')  

    if level == "HF":
        li = scf.UHF(mol).run() 
    elif level == "b3lyp":
        li = dft.UKS(mol) #unrestricted kohn-sham instead of hartree fock
        li.xc = 'b3lyp'
        li.kernel()
    elif level == "MP2":
        li = scf.UHF(mol).run() 
        li.MP2().run()
    elif level == "CISD":
        li = scf.UHF(mol).run() 
        li = li.CISD().run()
    elif level == "CCSD":
        li = scf.UHF(mol).run() 
        li = li.CCSD().run()

    if li.converged:
        return(li.e_tot)
    else:
        return("not converged")

levels = ["HF","b3lyp","MP2","CISD"]
H_energy = []
for level in levels:
   thisEne = calc_H_level(level,'6-31g','H',0,1)
   H_energy.append(thisEne)
   print("**Level: "+level+" energy ="+str( thisEne))

In [ ]:
print(H_energy)

# Ionization energy of atoms

The ionization energy is the energy that it takes to remove the electron.

* The first ionization energy of lithium will be = $E_{Li}$ - $E_{Li^+}$
* The second ionization energy of lithium will be = $E_{Li^+}$ - $E_{Li^{2+}}$

So one just need to calculate the energy of those different atoms and do the difference.
Notice that the spin state is different in each case.

* Li has 1s2 2s1, so one unpaired electron s=1/2; 2s+1=2; use the uhf method
* Li(+) has 1s2, and no unpaired electrons s=0; 2s+1=1; use the rhf method
* Li(2+) has 1s1, so one unpaired electron again s=1/2; 2s+1=2; use the uhf method

**Restricted and Unrestricted wavefunctions**

<img width="300" src="./restricted.png">



In [ ]:
from scipy import constants

hartree_value = 1  # Value in Hartree
electronvolt_value = hartree_value * constants.physical_constants["Hartree energy in eV"][0]

level = 'b3lyp'
basis = '6-31g'
atom = 'Li'
ene_Li_neutral = calc_H_level(level,basis,atom,0,1)
ene_Li_cation = calc_H_level(level,basis,atom,1,0)

ionE_Li = ene_Li_cation - ene_Li_neutral
print(ionE_Li*electronvolt_value)

In [ ]:
level = 'b3lyp'
basis = '6-31g'
atom = 'Be'
ene_Be_neutral = calc_H_level(level,basis,atom,0,0)
ene_Be_cation = calc_H_level(level,basis,atom,1,1)

ionE_Be = ene_Be_cation - ene_Be_neutral
print(ionE_Be*electronvolt_value)

# Questions

1. Calculate the 1st, 2nd and 3rd ionization energies of Beryllium, Boron, and Carbon. Pay special attention to the charge and spin multiplicity in each case. Remember to use uhf for unpaired electrons and rhf for paired electrons.
    1. Investigate the effect of using different methods: the scf and the b3lyp method.
    2. Investigate the effect of using different bassis sets: sto-3g and 6-31G**
2. Save the output of each calculation in a separate file. Be systematic in the naming so you can read it in a loop. Since you are submitting your assignments folder, create a subfolder called M1D4 with all these files.
3. Build a table containing ionization energies, number of basis sets in each calculation and time that it took to run the calculation. Do not do this manually, do it with a loop that opens each of the files. Start simple and grow in complexity checking every step.
4. Plot the ionization energies